In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/

In [ ]:
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch
import random
import matplotlib.pyplot as plt
import torch.backends.cudnn as cudnn
import queue
from IPython.display import display, Javascript, Image, clear_output
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode

from webcamdetect import video_frame, video_stream, js_to_image, bbox_to_bytes

In [16]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/gdrive/MyDrive/yolov5/UNOCards-object-detection/weights/best.pt')  # local repo

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-5-17 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 232 layers, 7284276 parameters, 0 gradients
Adding AutoShape... 


In [17]:
%cd /content/gdrive/MyDrive/yolov5/UNOCards-object-detection/

/content/gdrive/MyDrive/yolov5/UNOCards-object-detection


In [ ]:
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''
# Load the model and set the hyperparameters up
cudnn.benchmark = True
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.45  # NMS IoU threshold (0-1)
model.classes = None #[0, 15, 16]  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs

my_queue = queue.Queue()
def storeInQueue(f):
  def wrapper(*args):
    my_queue.put(f(*args))
  return wrapper

@storeInQueue
def detection(model, frame, bbox_array):
  result = model(frame)
  
  Class_name = result.pandas().xyxy[0].name
  Confidence = result.pandas().xyxy[0].confidence
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(Class_name))]
  Xmin = result.pandas().xyxy[0].xmin
  Ymin = result.pandas().xyxy[0].ymin
  Xmax = result.pandas().xyxy[0].xmax
  Ymax = result.pandas().xyxy[0].ymax
  
  _, _, h, w = result.s
  img_height, img_width, _ = frame.shape
  
  width_ratio = img_width/w
  height_ratio = img_height/h

  for i in range(0, len(Class_name)):
    xmin = int(Xmin[i] + 0.5)
    ymin = int(Ymin[i] + 0.5)
    xmax = int(Xmax[i] + 0.5)
    ymax = int(Ymax[i] + 0.5)

    xmin = int(xmin * width_ratio)
    ymin = int(ymin * height_ratio)
    xmax = int(xmax * width_ratio)
    ymax = int(ymax * height_ratio)

    bbox_array = cv2.rectangle(bbox_array, (xmin, ymin), (xmax, ymax), colors[i], 2)
    bbox_array = cv2.putText(bbox_array, Class_name[i],
                        (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        colors[i], 2)
  bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
  # convert overlay of bbox into bytes
  bbox_bytes = bbox_to_bytes(bbox_array)
  
  # update bbox so next frame gets new overlay
  # bbox = bbox_bytes
  return bbox_bytes



while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break
  
  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # create transparent overlay for bounding box
  bbox_array = np.zeros([480,640,4], dtype=np.uint8)

  t1 = threading.Thread(target=detection, args=(model, frame, bbox_array))
  t1.start()
  #t1.join()
  my_data = my_queue.get()
  bbox = my_data